# SQuAD-Multitask-QuestionAnswer-Generation

In [11]:
!nvidia-smi

Mon Mar 28 18:38:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
# Import packages

from typing import List, Dict
import tqdm.notebook as tq
from tqdm.notebook import tqdm
import json
import pandas as pd
import numpy as np
import spacy
import nltk
import re

nltk.download('punkt')
spacy = spacy.load('en_core_web_sm')
!pip install --quiet pytorch-lightning
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
!pip3 install --upgrade tensorflow-gpu
!pip3 install tensorflow-hub
import tensorflow as tf
import tensorflow_hub as hub
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
!pip install transformers
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
    )

from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

pl.seed_everything(20)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Global seed set to 20


20

Mounting google drive to get access to the model weights.

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load up the Squad 1.1 Dataset

In [14]:
train = pd.read_json('train-v1.1.json')
dev = pd.read_json('dev-v1.1.json')


# Creating a New data frame out of the current SQuAD data

taking up to 10 answers per context and putting them in seperated columns, contexts with less than 10 answers would have 'XXXXXXXXXXX' in their missing answer column

In [16]:



def CreateDFMultiQuestion(dataset):
    
  data=[]
  for article in (dataset['data']):
    title=article['title']
    for paragraph in article['paragraphs']:
      para=paragraph['context']
      if len(paragraph['qas'])>=10:
        answers=[]
        for i in range(10):
          answers.append(paragraph['qas'][i]['answers'][0]['text'])
        data.append([title,para,answers[0],answers[1],answers[2],answers[3],answers[4],answers[5],answers[6],answers[7],answers[8],answers[9]])
      elif len(paragraph['qas'])==9:
        answers=[]
        for i in range(9):
          answers.append(paragraph['qas'][i]['answers'][0]['text'])
        for i in range(1):
          answers.append('XXXXXXXXXXX')
        data.append([title,para,answers[0],answers[1],answers[2],answers[3],answers[4],answers[5],answers[6],answers[7],answers[8],answers[9]])
      elif len(paragraph['qas'])==8:
        answers=[]
        for i in range(8):
          answers.append(paragraph['qas'][i]['answers'][0]['text'])
        for i in range(2):
          answers.append('XXXXXXXXXXX')
        data.append([title,para,answers[0],answers[1],answers[2],answers[3],answers[4],answers[5],answers[6],answers[7],answers[8],answers[9]])

      elif len(paragraph['qas'])==7:
        answers=[]
        for i in range(7):
          answers.append(paragraph['qas'][i]['answers'][0]['text'])
        for i in range(3):
          answers.append('XXXXXXXXXXX')
        data.append([title,para,answers[0],answers[1],answers[2],answers[3],answers[4],answers[5],answers[6],answers[7],answers[8],answers[9]])

      elif len(paragraph['qas'])==6:
        answers=[]
        for i in range(6):
          answers.append(paragraph['qas'][i]['answers'][0]['text'])
        for i in range(4):
          answers.append('XXXXXXXXXXX')
        data.append([title,para,answers[0],answers[1],answers[2],answers[3],answers[4],answers[5],answers[6],answers[7],answers[8],answers[9]])
      elif len(paragraph['qas'])==5:
        answers=[]
        for i in range(5):
          answers.append(paragraph['qas'][i]['answers'][0]['text'])
        for i in range(5):
          answers.append('XXXXXXXXXXX')
        data.append([title,para,answers[0],answers[1],answers[2],answers[3],answers[4],answers[5],answers[6],answers[7],answers[8],answers[9]])

      elif len(paragraph['qas'])==4:
        answers=[]
        for i in range(4):
          answers.append(paragraph['qas'][i]['answers'][0]['text'])
        for i in range(6):
          answers.append('XXXXXXXXXXX')
        data.append([title,para,answers[0],answers[1],answers[2],answers[3],answers[4],answers[5],answers[6],answers[7],answers[8],answers[9]])

      elif len(paragraph['qas'])==3:
        answers=[]
        for i in range(3):
          answers.append(paragraph['qas'][i]['answers'][0]['text'])
        for i in range(7):
          answers.append('XXXXXXXXXXX')
        data.append([title,para,answers[0],answers[1],answers[2],answers[3],answers[4],answers[5],answers[6],answers[7],answers[8],answers[9]])

      elif len(paragraph['qas'])==2:
        answers=[]
        for i in range(2):
          answers.append(paragraph['qas'][i]['answers'][0]['text'])
        for i in range(8):
          answers.append('XXXXXXXXXXX')

        data.append([title,para,answers[0],answers[1],answers[2],answers[3],answers[4],answers[5],answers[6],answers[7],answers[8],answers[9]])

      elif len(paragraph['qas'])==1:
        answers=[]
        for i in range(1):
          answers.append(paragraph['qas'][i]['answers'][0]['text'])
        for i in range(9):
          answers.append('XXXXXXXXXXX')

        data.append([title,para,answers[0],answers[1],answers[2],answers[3],answers[4],answers[5],answers[6],answers[7],answers[8],answers[9]])



  dataframe=pd.DataFrame(data=data,columns=['title','context','ans1','ans2','ans3','ans4','ans5','ans6','ans7','ans8','ans9','ans10'])
  return dataframe


In [17]:
train_df=CreateDFMultiQuestion(train)
dev_df=CreateDFMultiQuestion(dev)

train_df = train_df.dropna() 
dev_df = dev_df.dropna() 
train_df=train_df.drop(columns=['title'])
dev_df=dev_df.drop(columns=['title'])

train_df = train_df.sample(frac=1,random_state=20).reset_index(drop=True)
test_df = train_df[:1000] #11877
train_df = train_df[1000:]

print(train_df.shape, 'train_df')
print(dev_df.shape, 'dev_df')
print(test_df.shape, 'test_df')

train_df.head()

(17896, 11) train_df
(2067, 11) dev_df
(1000, 11) test_df


,context,ans1,ans2,ans3,ans4,ans5,ans6,ans7,ans8,ans9,ans10
1000,An early advantage of AC is that the power-was...,multiple taps on the transformer,power-wasting resistors,low-voltage transformer windings,three-phase induction motor,can run equally well on DC or AC of any frequency,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX
1001,"""Green"" in modern European languages correspon...",Thai,520–570 nm,the name of a fruit,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX
1002,Almost all Protestant denominations are repres...,74,two,1814,Saint George's Anglican Church,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX
1003,Hurling and football are the most popular spec...,Hurling and football,30,7,Camogie,gaelic football,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX
1004,"From 1932, it was taught that the ""little floc...",1932,"144,000",a paradise restored on earth,great multitude,1914,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX,XXXXXXXXXXX


Encoding the data as input and target strings

In [21]:
def encodeData(model_name,row,tokenizer,SEP_TOKEN,ANS_TOKEN,max_length):
  
  string=""
  for i in range(1,len(row)):
    if row[i]!='XXXXXXXXXXX':
      string+=' '
      string+=row[i]
      string+=' '
      string+=ANS_TOKEN
  string+=SEP_TOKEN
  string+=' '
  string+=row['context']
  input_encoding = tokenizer(
        string,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
  return input_encoding

def encodeAnswer(model_name,row,tokenizer,SEP_TOKEN,ANS_TOKEN,target_max_token_len):
  string=""
  for i in range(1,len(row)):
    if row[i]!='XXXXXXXXXXX':
      string+=' '
      string+=row[i]
      string+=' '
      string+=ANS_TOKEN
 
  target_encoding = tokenizer(
         string,
        max_length=target_max_token_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
 
  labels=target_encoding['input_ids']
  labels[labels==0]= -100
  return target_encoding,labels

# Creating a Dataset class 
this class also handles the masking process for training.

In [22]:
class AnswerGenDataset(Dataset):
    
    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: T5Tokenizer,
        data_max_token_len:int= 500,
        target_max_token_len: int= 250,
        SEP_TOKEN:str='<<sep>>',
        ANS_TOKEN:str='<<ans>>',
        model:str='t5-small'

        ):

        self.tokenizer = tokenizer
        self.data = data
        self.data_max_token_len = data_max_token_len
        self.target_max_token_len = target_max_token_len
        self.SEP_TOKEN='<<sep>>'
        self.ANS_TOKEN='<<ans>>'
        self.model='t5-small'


    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        row = self.data.iloc[index]
        
        maskProb=0.4
        targetRow=row.copy()
        if np.random.rand() <= maskProb:
            row['ans1'] = '@MASK@'
        if np.random.rand() <= maskProb and row['ans2']!='XXXXXXXXXXX':
            row['ans2'] = '@MASK@'
        if np.random.rand() <= maskProb and row['ans3']!='XXXXXXXXXXX':
            row['ans3'] = '@MASK@'
        if np.random.rand() <= maskProb and row['ans4']!='XXXXXXXXXXX':
            row['ans4'] = '@MASK@'
        if np.random.rand() <= maskProb and row['ans5']!='XXXXXXXXXXX':
            row['ans5'] = '@MASK@'
        maskProb=0.2
        if np.random.rand() <= maskProb and row['ans6']!='XXXXXXXXXXX':
            row['ans6'] = '@MASK@'
        if np.random.rand() <= maskProb and row['ans7']!='XXXXXXXXXXX':
            row['ans7'] = '@MASK@'
        if np.random.rand() <= maskProb and row['ans8']!='XXXXXXXXXXX':
            row['ans8'] = '@MASK@'
        if np.random.rand() <= maskProb and row['ans9']!='XXXXXXXXXXX':
            row['ans9'] = '@MASK@'
        if np.random.rand() <= maskProb and row['ans10']!='XXXXXXXXXXX':
            row['ans10'] = '@MASK@'

        input_encoding=encodeData(self.model,row,tokenizer,self.SEP_TOKEN,self.ANS_TOKEN,self.data_max_token_len)
        target_encoding,labels=encodeAnswer(self.model,targetRow,tokenizer,self.SEP_TOKEN,self.ANS_TOKEN,self.target_max_token_len)


        return dict(
            answer1 = row['ans1'],
            answer2 = row['ans2'],
            answer3 = row['ans3'],
            answer4 = row['ans4'],
            answer5 = row['ans5'],
            answer6 = row['ans6'],
            answer7 = row['ans7'],
            answer8 = row['ans8'],
            answer9 = row['ans9'],
            answer10 = row['ans10'],
            context = row['context'],
            input_ids = input_encoding['input_ids'].flatten(),
            attention_mask = input_encoding['attention_mask'].flatten(),
            labels=labels.flatten()
            )
        

    

Creating the Dataset

In [23]:
model_name=model='t5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
SEP_TOKEN='<<sep>>'
ANS_TOKEN='<<ans>>'
tokenizer.add_tokens(SEP_TOKEN)
tokenizer.add_tokens(ANS_TOKEN)
TOKENIZER_LEN = len(tokenizer)
print(TOKENIZER_LEN)
sample_dataset=AnswerGenDataset(train_df,tokenizer)



Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

32102


Taking a Peek inside the dataset

In [25]:
sample_dataset[66]

{'answer1': 'Mac OS, System 7',
 'answer10': 'XXXXXXXXXXX',
 'answer2': 'Frogdesign',
 'answer3': 'the Apple Industrial Design Group',
 'answer4': 'Apple Industrial Design Group',
 'answer5': '@MASK@',
 'answer6': 'XXXXXXXXXXX',
 'answer7': 'XXXXXXXXXXX',
 'answer8': 'XXXXXXXXXXX',
 'answer9': 'XXXXXXXXXXX',
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 

Creating a Pl data module

In [26]:
class AnswerGenDataModule(pl.LightningDataModule):

    def __init__(
        self,
        train_df: pd.DataFrame,
        val_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: T5Tokenizer,
        batch_size: int =16,
        data_max_token_len:int= 500,
        target_max_token_len: int= 250

        ): 
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.data_max_token_len = data_max_token_len
        self.target_max_token_len = target_max_token_len

    def setup(self):
        self.train_dataset = AnswerGenDataset(self.train_df, self.tokenizer, self.data_max_token_len, self.target_max_token_len)
        self.val_dataset = AnswerGenDataset(self.val_df, self.tokenizer, self.data_max_token_len, self.target_max_token_len)
        self.test_dataset = AnswerGenDataset(self.test_df, self.tokenizer, self.data_max_token_len, self.target_max_token_len)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size = self.batch_size, shuffle=True, num_workers = 2)

    def val_dataloader(self): 
        return DataLoader(self.val_dataset, batch_size=1, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=1, num_workers=2)

Setting up some hyper parameters

In [27]:
BATCH_SIZE = 16
N_EPOCHS = 10
LEARNING_RATE = 0.0001
MODEL_NAME = 't5-small'
DATA_MAX_TOKEN_LEN = 500
TARGET_MAX_TOKEN_LEN = 250

data_module = AnswerGenDataModule(train_df, dev_df, test_df, tokenizer, BATCH_SIZE,DATA_MAX_TOKEN_LEN,TARGET_MAX_TOKEN_LEN)
data_module.setup()

Creating a PL model class

In [28]:
class AGModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
        self.model.resize_token_embeddings(TOKENIZER_LEN) #resizing after adding new tokens to the tokenizer

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('train_loss', loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('test_loss', loss, prog_bar=True, logger=True)
        return loss
  
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=LEARNING_RATE)

In [29]:
checkpoint_callback = ModelCheckpoint(
        dirpath='checkpoints',
        filename='best-checkpoint',
        save_top_k=-1,
        verbose=True,
        monitor='val_loss',
        mode='min')
trainer = pl.Trainer(
        checkpoint_callback= checkpoint_callback,
        max_epochs=N_EPOCHS,
        gpus=1,
        progress_bar_refresh_rate=30)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:148: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f3ba2377f10>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f3ba2377f10>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Tr

# Loading up tensorboard and start training

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

In [ ]:
model = AGModel()
trainer.fit(model, data_module)

Loading up the trained weights, can choose new train weights or the old one.


In [30]:
checkpoint_path = '/content/drive/MyDrive/AnswerGen/AnswerGenWeights.ckpt'
best_model = AGModel.load_from_checkpoint(checkpoint_path)
best_model.freeze()
best_model.eval()

print()

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Generate functions to generate answers from our T5 model

In [31]:
def generate_answerNoraml(AGModel: AGModel ,context,max_length,num_beams) :# The first Function to generate up to 10 answers per context
    SEP_TOKEN='<<sep>>'
    ANS_TOKEN='<<ans>>'
    string=""
    for i in range(10): # stringing together 10 Mask tokens
      string+=' '
      string+='@MASK@'
      string+=' '
      string+=ANS_TOKEN
    string+=SEP_TOKEN
    string+=' '
    string+=context
    source_encoding = tokenizer(
        string,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    generated_ids = AGModel.model.generate(
        input_ids=source_encoding['input_ids'],
        attention_mask=source_encoding['attention_mask'],
        num_beams=num_beams,
        max_length=250,
        repetition_penalty=5.0,
        no_repeat_ngram_size=3,
        length_penalty=5.0,
        early_stopping=True,
        use_cache=True
    )

    preds = {
        tokenizer.decode(generated_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    }

    return ''.join(preds)
    
def generate_answerWithMinLength(AGModel: AGModel ,context,max_length,num_beams,min_length) : # the second function that will be used to generate an increased amount of answers per text
    SEP_TOKEN='<<sep>>'
    ANS_TOKEN='<<ans>>'
    string=""
    for i in range(10): # stringing together 10 Mask tokens
      string+=' '
      string+='@MASK@'
      string+=' '
      string+=ANS_TOKEN
    string+=SEP_TOKEN
    string+=' '
    string+=context
    source_encoding = tokenizer(
        string,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    generated_ids = AGModel.model.generate(
        input_ids=source_encoding['input_ids'],
        attention_mask=source_encoding['attention_mask'],
        num_beams=num_beams,
        max_length=250,
        min_length=min_length,
        repetition_penalty=5.0,
        no_repeat_ngram_size=3,
        length_penalty=5.0,
        early_stopping=True,
        use_cache=True
    )
    preds = {
        tokenizer.decode(generated_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    }
    return ''.join(preds)

Helper functions to clean the generated text

In [32]:
def GeneAnswersToListNormal(best_model,context,len,num_beams):# an helper function to remove tokens that were created during the generation procces
  ans=generate_answerNoraml(best_model,context,len,num_beams)
  ans=ans.replace('<pad>','')
  answer=''
  answers=[]
  flag=0
  for word in ans:
    if word=='<':
      flag=1
    elif word =='>':
      flag=0
      if answer!='':
        answers.append(answer)
        answer=''
    else:
      if flag==0:
        answer=answer + word
  return answers
  
def GeneAnswersToListMinLength(best_model,context,len,num_beams,min_length): #an helper function to remove tokens that were created during the generation procces this time using the min_length parameter
  ans=generate_answerWithMinLength(best_model,context,len,num_beams,min_length)
  ans=ans.replace('<pad>','')
  answer=''
  answers=[]
  flag=0
  for word in ans:
    if word=='<':
      flag=1
    elif word =='>':
      flag=0
      if answer!='':
        answers.append(answer)
        answer=''
    else:
      if flag==0:
        answer=answer + word
  return answers


# Generating example on a random Wikipedia text about Bears

In [ ]:

context="""Bears are carnivoran mammals of the family Ursidae. They are classified as caniforms, or doglike carnivorans. Although only eight species of bears are extant, they are widespread, appearing in a wide variety of habitats throughout the Northern Hemisphere and partially in the Southern Hemisphere. Bears are found on the continents of North America, South America, Europe, and Asia. Common characteristics of modern bears include large bodies with stocky legs, long snouts, small rounded ears, shaggy hair, plantigrade paws with five nonretractile claws, and short tails While the polar bear is mostly carnivorous, and the giant panda feeds almost entirely on bamboo, the remaining six species are omnivorous with varied diets. With the exception of courting individuals and mothers with their young, bears are typically solitary animals. They may be diurnal or nocturnal and have an excellent sense of smell. Despite their heavy build and awkward gait, they are adept runners, climbers, and swimmers. Bears use shelters, such as caves and logs, as their dens; most species occupy their dens during the winter for a long period of hibernation, up to 100 days.Bears have been hunted since prehistoric times for their meat and fur; they have been used for bear-baiting and other forms of entertainment, such as being made to dance. With their powerful physical presence, they play a prominent role in the arts, mythology, and other cultural aspects of various human societies. In modern times, bears have come under pressure through encroachment on their habitats and illegal trade in bear parts, including the Asian bile bear market. The IUCN lists six bear species as vulnerable or endangered, and even least concern species, such as the brown bear, are at risk of extirpation in certain countries. The poaching and international trade of these most threatened populations are prohibited, but still ongoing. The English word bear comes from Old English bera and belongs to a family of names for the bear in Germanic languages, such as Swedish björn, also used as a first name. This form is conventionally said to be related to a Proto-Indo-European word for brown, so that bear would mean the brown one.[1][2] However, Ringe notes that while this etymology is semantically plausible, a word meaning brown of this form cannot be found in Proto-Indo-European. He suggests instead that bear is from the Proto-Indo-European word *ǵʰwḗr- ~ *ǵʰwér wild animal.[3] This terminology for the animal originated as a taboo avoidance term: proto-Germanic tribes replaced their original word for bear—arkto—with this euphemistic expression out of fear that speaking the animal's true name might cause it to appear.[4][5] According to author Ralph Keyes, this is the oldest known euphemism.[6] Bear taxon names such as Arctoidea and Helarctos come from the ancient Greek ἄρκτος (arktos), meaning bear,[7] as do the names arctic and antarctic, via the name of the constellation Ursa Major, the Great Bear, prominent in the northern sky.[8] Bear taxon names such as Ursidae and Ursus come from Latin Ursus/Ursa, he-bear/she-bear.[8] The female first name Ursula, originally derived from a Christian saint's name, means little she-bear (diminutive of Latin ursa). In Switzerland, the male first name Urs is especially popular, while the name of the canton and city of Bern is derived from Bär, German for bear. The Germanic name Bernard (including Bernhardt and similar forms)"""
answersList=GeneAnswersToListNormal(best_model,context,500,10)
print(answersList)

 Functions to generate answers on a dataframe of contexts, this output is gonna go to Doron Model to create questions.

In [ ]:

def GeneAnswersForDFToDFWithMaxAnswers(best_model,dataset,max_length,max_answers,frac=0.005):

  contextAnswerList=[]
  dataset = dataset.sample(frac=frac,random_state=20).reset_index(drop=True)
  for context in dataset['context']:
    LessThanXFlag=True
    min_length=10
    num_beams=10
    firstTime=True
    while LessThanXFlag:
      if firstTime:
        answersSet=set()
        answersCheckDuplicateSet=set()
        firsTime=False

      answersList=GeneAnswersToListMinLength(best_model,context,max_length,num_beams,min_length) #generate Answers with a required amount of answers
      answersListOrginial=answersList.copy()
      
      for i in range(len(answersList)):  #making sure our answers are unique and we are not getting double answers like "The Cat", "the cat", "The cat."
          answersList[i]=answersList[i].lower()
          answersList[i] = re.sub("""[!@#()-+$&^]['":,.]""", '', answersList[i])
          if answersList[i] not in answersCheckDuplicateSet and answersList[i]!="mask":
            answersSet.add(answersListOrginial[i])
            if len(answersSet)==max_answers:
               LessThanXFlag=False
               break
          answersCheckDuplicateSet.add(answersList[i])
        
      if len(answersSet)==max_answers:
          LessThanXFlag=False
      else:
        min_length+=40

    for answer in answersSet:
        contextAnswerList.append([context,answer])
  dataframe=pd.DataFrame(data=contextAnswerList,columns=['context','answer'])
  return dataframe    
    
      

In [ ]:
def GeneAnswersForDFToDF(best_model,dataset,max_length,frac=0.05):
  

  answersSet=set()
  answersCheckDuplicateSet=set()
  contextAnswerList=[]
  dataset = dataset.sample(frac=frac,random_state=20).reset_index(drop=True)
  for context in dataset['context']:
    num_beams=10
    answersList=GeneAnswersToListNormal(best_model,context,max_length,num_beams) #generate up to 10 answers per context
    answersListOrginial=answersList.copy()
    answersSet=set()
    answersCheckDuplicateSet=set()
    for i in range(len(answersList)):  #making sure our answers are unique and we are not getting double answers like "The Cat", "the cat", "The cat."
          answersList[i]=answersList[i].lower()
          answersList[i] = re.sub("""[!@#()-+$&^]['":,.]""", '', answersList[i])

          if answersList[i] not in answersCheckDuplicateSet and answersList[i]!="mask":
            answersSet.add(answersListOrginial[i])
          answersCheckDuplicateSet.add(answersList[i])

    
    for answer in answersSet:
        contextAnswerList.append([context,answer])
  dataframe=pd.DataFrame(data=contextAnswerList,columns=['context','answer'])
  return dataframe


In [ ]:
contextAnswer_df=GeneAnswersForDFToDF(best_model,test_df,500)
contextAnswer_df.to_csv('contextAnswer_df.csv',encoding='utf-8', index=False)
contextAnswer_df


# create dataset For Metrics
making sure we have the same amount of labels from each class. (generated or non-generated) 

This functions takes a lot of time hence sampling only a fraction of the data.


In [33]:
def CreateNotGenDF(dataset):

  contextAnswerNotGenList=[]

  for i in range(len(dataset)):
    for j in range (1,11):
      ans=dataset.iloc[i][j]
      if ans!='XXXXXXXXXXX':
        contextAnswerNotGenList.append([dataset.iloc[i][0],ans,'0'])

  dataframe=pd.DataFrame(data=contextAnswerNotGenList,columns=['context','answer','label'])
  return dataframe

In [40]:

def GeneAnswersForDFToDFToMetrics(best_model,dataset,max_length):

  answersSet=set()
  answersCheckDuplicateSet=set()
  contextAnswerList=[]
  datasetForNotGen=dataset.copy()
  dataset = dataset.sample(frac=0.2,random_state=20).reset_index(drop=True)
  for context in dataset['context']:
    num_beams=10
    answersList=GeneAnswersToListNormal(best_model,context,max_length,num_beams)
    answersListOrginial=answersList.copy()
    answersSet=set()
    answersCheckDuplicateSet=set()
    for i in range(len(answersList)):
          answersList[i]=answersList[i].lower()
          answersList[i] = re.sub("""[!@#()-+$&^]['":,.]""", '', answersList[i])
          if answersList[i] not in answersCheckDuplicateSet and answersList[i]!="mask":
            answersSet.add(answersListOrginial[i])
          answersCheckDuplicateSet.add(answersList[i])
    
    
    for answer in answersSet:
        contextAnswerList.append([context,answer,'1'])
  print(len(contextAnswerList)) #printing the amount of answers of one label
  NotGen_df=CreateNotGenDF(datasetForNotGen)
  NotGen_df = NotGen_df.sample(frac=1.0,random_state=20).reset_index(drop=True)

  contextAnswerListLength=len(contextAnswerList)
  
  for i in range(contextAnswerListLength):
    contextAnswerList.append([NotGen_df.iloc[i][0],NotGen_df.iloc[i][1],NotGen_df.iloc[i][2]])
  
    
  dataframe=pd.DataFrame(data=contextAnswerList,columns=['context','answer','label'])
  return dataframe

# Using the function to generate Metrics DF for the metric notebook
contextAnswer_df=GeneAnswersForDFToDFToMetrics(best_model,train_df,500)
contextAnswer_df.to_csv('contextAnswerfORROBERTA_df.csv',encoding='utf-8', index=False)
contextAnswer_df

    
path = '/content/drive/MyDrive/AnswerGenMetrics.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  contextAnswer_df.to_csv(f)
    
      


29134


In [ ]:

def GeneAnswersForDFToDFWithMaxAnswersForMetrics(best_model,dataset,max_length,max_answers): # create a Dataframe of [context, answers] pairs

  answersSet=set()
  answersCheckDuplicateSet=set()
  contextAnswerList=[]
  datasetForNotGen=dataset.copy()

  dataset = dataset.sample(frac=0.2,random_state=20).reset_index(drop=True)
  for context in dataset['context']:
    LessThanXFlag=True
    min_length=10
    num_beams=10
    firstTime=True
    while LessThanXFlag:
      if firstTime:
        answersSet=set()
        answersCheckDuplicateSet=set()
        firsTime=False

      answersList=GeneAnswersToListMinLength(best_model,context,max_length,num_beams,min_length) #generate Answers with a required amount of answers
      answersListOrginial=answersList.copy()
      
      for i in range(len(answersList)): #making sure our answers are unique and we are not getting double answers like "The Cat", "the cat", "The cat."
          answersList[i]=answersList[i].lower()
          answersList[i] = re.sub("""[!@#()-+$&^]['":,.]""", '', answersList[i])
          if answersList[i] not in answersCheckDuplicateSet and answersList[i]!="mask":
            answersSet.add(answersListOrginial[i])
            if len(answersSet)==max_answers:
               LessThanXFlag=False
               break
          answersCheckDuplicateSet.add(answersList[i])
        
      if len(answersSet)==max_answers:
          LessThanXFlag=False
      else:
        min_length+=40

    
    for answer in answersSet:
        contextAnswerList.append([context,answer,'1'])
  print(len(contextAnswerList)) #printing the amount of answers of one label
  NotGen_df=CreateNotGenDF(datasetForNotGen)
  NotGen_df = NotGen_df.sample(frac=1.0,random_state=20).reset_index(drop=True)

  contextAnswerListLength=len(contextAnswerList)
  
  for i in range(contextAnswerListLength):
    contextAnswerList.append([NotGen_df.iloc[i][0],NotGen_df.iloc[i][1],NotGen_df.iloc[i][2]])
  
    
  dataframe=pd.DataFrame(data=contextAnswerList,columns=['context','answer','label'])
  return dataframe
    
      


Yay you reached the end! Now continue with the next step of our system, question generation!